# 形態素解析の実装

Transformerによるテキストデータのポジネガ分析  
まずは文章を単語に分割，すなわち形態素解析を，以下のツールを使用して行う  
- Janome
- MeCab
- NEologd

## 必要なデータセットのダウンロード

In [1]:
import os
import urllib.request
import zipfile
import tarfile

In [4]:
data_dir = "../../datasets/ptca_datasets/chapter7"
if not os.path.exists(data_dir):
    os.mkdir(data_dir)

### word2vec学習済みモデルをダウンロード

東北大学 乾・岡崎研究室で[公開](http://www.cl.ecei.tohoku.ac.jp/~m-suzuki/jawiki_vector/)されている[データ](http://www.cl.ecei.tohoku.ac.jp/~m-suzuki/jawiki_vector/data/20170201.tar.bz2)をダウンロードして使用  
  
日本語Wikipediaにある単語を200次元のword2vecにしたデータセット

保存先フォルダが存在しない場合は作成する

データのダウンロードを開始．15分ほどかかる

In [3]:
url = "http://www.cl.ecei.tohoku.ac.jp/~m-suzuki/jawiki_vector/data/20170201.tar.bz2"
save_path = os.path.join(data_dir, "20170201.tar.bz2")
if not os.path.exists(save_path):
    urllib.request.urlretrieve(url, save_path)

20170201.tar.bz2の解凍　5分ほどかかる  
entity_vectorディレクトリと，その中にentity_vector.model.binができる

In [4]:
tar = tarfile.open(save_path, 'r|bz2') # tarファイルを読み込み
tar.extractall(data_dir)  # 解凍
tar.close()
os.remove(save_path)

### IMDbデータセットをダウンロード

映画評価データセットの[IMDbデータセット](http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz)をダウンロード 30秒ほどかかる

In [6]:
url = "http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"
save_path = os.path.join(data_dir, "aclImdb_v1.tar.gz")
if not os.path.exists(save_path):
    urllib.request.urlretrieve(url, save_path)

aclImdb_v1.tar.gzの解凍 1分ほどかかる  
aclImdbディレクトリができる

In [8]:
tar = tarfile.open(save_path)
tar.extractall(data_dir)
tar.close()
os.remove(save_path)

### fastTextの英語学習済みモデルをダウンロード

fastTextの公式の英語学習済みモデル（650MB）をダウンロード 5分ほどかかる  

In [9]:
url = "https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip"
save_path = os.path.join(data_dir, "wiki-news-300d-1M.vec.zip")
if not os.path.exists(save_path):
    urllib.request.urlretrieve(url, save_path)

wiki-news-300d-1M.vec.zipを解凍  
wiki-news-300d-1M.vecファイルができる

In [11]:
file = zipfile.ZipFile(save_path)
file.extractall(data_dir)
file.close()
os.remove(save_path)

### fastTextの日本語学習済みモデルをダウンロード
- [gdriveコマンド](https://takuya-1st.hatenablog.jp/entry/2016/07/06/034412)を使えるようにする  
- data_dirへ行き，`gdrive download 0ByFQ96A4DgSPUm9wVWRLdm5qbmc`を実行
    - tokenを求められるのでブラウザで取得
- 次のセルのコードでzipを解凍
    - model.vecが得られる

参考：[Qiita 「いますぐ使える単語埋め込みベクトルのリスト」](https://qiita.com/Hironsan/items/8f7d35f0a36e0f99752c)

In [3]:
save_path = os.path.join(data_dir, "vector_neologd.zip")
file = zipfile.ZipFile(save_path)
file.extractall(data_dir)
file.close()
os.remove(save_path)

### 著者によって用意されたファイル
masterにある，chapter7_.../data内のファイル  
- text_test.tsv
- text_train.tsv
- text_val.tsv

をdata_dirへ移動しておく

## 機械学習における自然言語処理の流れ
- コーパスの収集: 集めた文書データのこと  
- クリーニング: HTMLやヘッダーといった文章ではないノイズ部分を削除すること
- 正規化： 全角半角の統一，大文字小文字の統一，表記ゆれ(ex. 猫，ネコ，ねこ)の統一
- 単語分割(形態素解析): 単語ごとに文章を区切る
- 単語の基本形への変換: 原形，語幹への変換．ex. 走っ→走る
- ストップワードの除去: 助詞など意味をあまり持たない単語を削除する
- 単語の数値化: 単語にIDを振ったり，ベクトル化する

## Janomeによる単語分割
Janomeは日本語文章の単語分割(形態素解析)を行うパッケージ  
Janomeをインストール
``` bash
$ pip insatll janome
```

In [2]:
from janome.tokenizer import Tokenizer

j_t = Tokenizer()

text = '私は機械学習が好きです。'

for token in j_t.tokenize(text):
    print(token)

# 分割結果だけが欲しい時には，wakati=Trueを与える
print(j_t.tokenize(text, wakati=True))

私	名詞,代名詞,一般,*,*,*,私,ワタシ,ワタシ
は	助詞,係助詞,*,*,*,*,は,ハ,ワ
機械	名詞,一般,*,*,*,*,機械,キカイ,キカイ
学習	名詞,サ変接続,*,*,*,*,学習,ガクシュウ,ガクシュー
が	助詞,格助詞,一般,*,*,*,が,ガ,ガ
好き	名詞,形容動詞語幹,*,*,*,*,好き,スキ,スキ
です	助動詞,*,*,*,特殊・デス,基本形,です,デス,デス
。	記号,句点,*,*,*,*,。,。,。
['私', 'は', '機械', '学習', 'が', '好き', 'です', '。']


## MeCab+NEologdによる単語分割
MeCabも単語分割(形態素解析)のライブラリ．  
MeCabは，清語辞書であるNEologdと合わせて使用することができる．  
  
MeCabのインストール
``` bash
sudo apt install mecab libmecab-dev mecab-ipadic-utf8 -y
pip install mecab-python3
```

NEologdのインストール
``` bash
git clone https://github.com/neologd/mecab-ipadic-neologd.git
cd mecab-ipadic-neologd
sudo bin/install-mecab-ipadic-neologd
```
インストールするか尋ねられ，yesと答えらインストール先ディレクトリが表示されるのでコピーしておく

In [1]:
import MeCab

m_t = MeCab.Tagger('-Ochasen')

text = '私は機械学習が好きです。'

print(m_t.parse(text))

私	ワタシ	私	名詞-代名詞-一般		
は	ハ	は	助詞-係助詞		
機械	キカイ	機械	名詞-一般		
学習	ガクシュウ	学習	名詞-サ変接続		
が	ガ	が	助詞-格助詞-一般		
好き	スキ	好き	名詞-形容動詞語幹		
です	デス	です	助動詞	特殊・デス	基本形
。	。	。	記号-句点		
EOS



NEologdを使うことで，「機械」と「学習」に分かれていた言葉が「機械学習」という単語になる  
さっきコピーして置いたneologdのインストール先ディレクトリへのパスを使う

In [2]:
m_t = MeCab.Tagger('-Ochasen -d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd')
print(m_t.parse(text))

私	ワタシ	私	名詞-代名詞-一般		
は	ハ	は	助詞-係助詞		
機械学習	キカイガクシュウ	機械学習	名詞-固有名詞-一般		
が	ガ	が	助詞-格助詞-一般		
好き	スキ	好き	名詞-形容動詞語幹		
です	デス	です	助動詞	特殊・デス	基本形
。	。	。	記号-句点		
EOS



MeCab+NEologdによる文章の単語リストへの分割関数を作成

In [5]:
import MeCab

def tokenizer_mecab(text, neologd_path='/usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd'):
    m_t = MeCab.Tagger(f'-Owakati -d {neologd_path}') # wakatiを使って品詞情報をなくす
    text = m_t.parse(text)
    return text.strip().split()

text = '私は機械学習が好きです。'
print(tokenizer_mecab(text))

['私', 'は', '機械学習', 'が', '好き', 'です', '。']
